In [1]:
import keras

In [2]:
from keras.optimizers import RMSprop
from keras.preprocessing.image import ImageDataGenerator
import cv2
from keras.models import Sequential
from keras.layers import Conv2D, Input, ZeroPadding2D, BatchNormalization, Activation, MaxPooling2D, Flatten, Dense,Dropout
from keras.models import Model, load_model
from keras.callbacks import TensorBoard, ModelCheckpoint
from sklearn.model_selection import train_test_split
from sklearn.metrics import f1_score
from sklearn.utils import shuffle
import imutils
import numpy as np

In [14]:
model = Sequential([
    Conv2D(100, (3,3), activation='relu', input_shape=(150, 150, 3)),
    MaxPooling2D(2,2),
    
    Conv2D(100, (3,3), activation='relu'),
    MaxPooling2D(2,2),
    
    Flatten(),
    Dropout(0.5),
    Dense(50, activation='relu'),
    Dense(2, activation='softmax')
])
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['acc'])

In [15]:

TRAINING_DIR = "/content/facemask/dataset/train"
train_datagen = ImageDataGenerator(rescale=1.0/255,
                                   rotation_range=40,
                                   width_shift_range=0.2,
                                   height_shift_range=0.2,
                                   shear_range=0.2,
                                   zoom_range=0.2,
                                   horizontal_flip=True,
                                   fill_mode='nearest')

train_generator = train_datagen.flow_from_directory(TRAINING_DIR, 
                                                    batch_size=10, 
                                                    target_size=(150, 150))
VALIDATION_DIR = "/content/facemask/dataset/test"
validation_datagen = ImageDataGenerator(rescale=1.0/255)

validation_generator = validation_datagen.flow_from_directory(VALIDATION_DIR, 
                                                         batch_size=10, 
                                                         target_size=(150, 150))




Found 1315 images belonging to 2 classes.
Found 194 images belonging to 2 classes.


In [16]:
train_generator.class_indices
validation_generator.class_indices

{'with_mask': 0, 'without_mask': 1}

In [17]:
from PIL import ImageFile
ImageFile.LOAD_TRUNCATED_IMAGES = True

In [11]:
import os
dir = os.listdir('/content/facemask/dataset/train')
print(dir)
file_path = ('/content/facemask/dataset/train/.ipynb_checkpoints')
os.rmdir(file_path)


['without_mask', 'with_mask', '.ipynb_checkpoints']


In [12]:
 import os
 dir = os.listdir('/content/facemask/dataset/train')
 print(dir)

['without_mask', 'with_mask']


In [18]:
checkpoint = ModelCheckpoint('model2-{epoch:03d}.model',
                             monitor='val_loss',verbose=0,
                             save_best_only=True,mode='auto')

In [19]:
history = model.fit_generator(train_generator,
                              epochs=10,
                              validation_data=validation_generator,
                              callbacks=[checkpoint])



/usr/local/lib/python3.6/dist-packages/tensorflow/python/keras/engine/training.py:1844: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  warnings.warn('`Model.fit_generator` is deprecated and '


Epoch 1/10
132/132 [==============================] - 118s 884ms/step - loss: 0.7775 - acc: 0.5820 - val_loss: 0.2280 - val_acc: 0.9227
INFO:tensorflow:Assets written to: model2-001.model/assets
Epoch 2/10
132/132 [==============================] - 115s 872ms/step - loss: 0.3349 - acc: 0.8505 - val_loss: 0.1340 - val_acc: 0.9588
INFO:tensorflow:Assets written to: model2-002.model/assets
Epoch 3/10
132/132 [==============================] - 116s 879ms/step - loss: 0.2720 - acc: 0.9008 - val_loss: 0.1746 - val_acc: 0.9691
Epoch 4/10
132/132 [==============================] - 115s 871ms/step - loss: 0.2869 - acc: 0.8810 - val_loss: 0.1826 - val_acc: 0.9124
Epoch 5/10
132/132 [==============================] - 116s 878ms/step - loss: 0.2293 - acc: 0.9143 - val_loss: 0.1186 - val_acc: 0.9691
INFO:tensorflow:Assets written to: model2-005.model/assets
Epoch 6/10
132/132 [==============================] - 116s 876ms/step - loss: 0.2287 - acc: 0.9136 - val_loss: 0.1050 - val_acc: 0.9485
INFO:te

In [20]:
model.save("mask_model.h5")


In [21]:
model.evaluate(train_generator)

132/132 [==============================] - 37s 282ms/step - loss: 0.2254 - acc: 0.9148


[0.22542501986026764, 0.914828896522522]

In [22]:
model.evaluate(validation_generator)

20/20 [==============================] - 4s 207ms/step - loss: 0.0816 - acc: 0.9794


[0.08163958042860031, 0.9793814420700073]

In [23]:
import os
model=load_model('mask_model.h5')


In [24]:
results={0:'without mask',1:'mask'}
GR_dict={0:(0,0,255),1:(0,255,0)}

In [39]:
haarcascade = cv2.CascadeClassifier('/content/facemask/haarcascade_frontalface_default.xml')